In [1]:
TRAINING_DATASET = '/data/patch_features_small/train2014.p'
TEST_DATASET = '/data/patch_features_small/test2014.p'
S3_BUCKET = 'cse-547'
S3_LOG_KEY = 'hw3/train/training_log_3a980bcc-3375-470c-a78c-10bcad23d268.pkl' # See train_s3_model_key.
S3_MODEL_KEY = 'hw3/train/model_3a980bcc-3375-470c-a78c-10bcad23d268.pkl' # See train_s3_model_key.

In [2]:
import pickle
from cse547.data import CocoPatchesDataset

with open(TEST_DATASET, 'rb') as f:
    dataset = CocoPatchesDataset.from_state_dict(pickle.load(f))

In [3]:
from cse547.models import LinearClassifier, MultiLayerPerceptron
from cse547.s3 import deserialize_object

training_log = deserialize_object(S3_BUCKET, S3_LOG_KEY)
#model = LinearClassifier(
#    n_features=dataset[0]['features'].size,
#    n_classes=dataset[0]['label'].size()[0])
model = MultiLayerPerceptron(
    n_features=dataset[0]['features'].size,
    n_classes=dataset[0]['label'].size()[0],
    hidden_units=training_log['model']['hidden_units'])
for param in model.parameters():
    param.requires_grad = False
model.load_state_dict(deserialize_object(S3_BUCKET, S3_MODEL_KEY))

In [4]:
import torch
from torch.autograd import Variable
from torch.nn import functional

from cse547.data import OneShotDataLoader

data_dict = iter(OneShotDataLoader(dataset)).next()
features = Variable(data_dict['features'])
labels = Variable(data_dict['label'])
with torch.no_grad():
    predictions = functional.sigmoid(model(features))

In [5]:
from sklearn.metrics import average_precision_score

test_average_precision_score = average_precision_score(labels.data.numpy(), predictions.data.numpy())
with open('test_average_precision_score.tex', 'w') as f:
    f.write('{0:.6}'.format(test_average_precision_score))
test_average_precision_score

0.39674770055797853

In [6]:
import numpy as np

with open(TRAINING_DATASET, 'rb') as f:
    training_observations_cnt = np.sum(
        iter(
            OneShotDataLoader(
                CocoPatchesDataset.from_state_dict(
                    pickle.load(f)))).next()['label'].numpy(),
        axis=0, dtype=np.int)

In [7]:
from collections import OrderedDict
import pandas as pd

evaluation_by_label = pd.DataFrame(OrderedDict([
    ('Label', [category['name'] for category in dataset.categories]),
    ('Training Observations', training_observations_cnt),
    ('Test Observations', np.sum(labels.data.numpy(), axis=0, dtype=np.int)),    
    ('Average Precision Score', average_precision_score(labels.data.numpy(), predictions.data.numpy(), average=None)),
]))
evaluation_by_label

,Label,Training Observations,Test Observations,Average Precision Score
0,bicycle,2166,416,0.140143
1,car,12179,2476,0.476441
2,motorcycle,2630,614,0.381392
3,airplane,3045,447,0.500307
4,bus,2972,543,0.313419
5,train,873,160,0.224836
6,truck,6058,1238,0.215611
7,boat,2961,650,0.262890
8,bird,6485,1473,0.428072
9,cat,2516,569,0.514017


In [8]:
with open('test_average_precision_score_by_class.tex', 'w') as f:
    f.write(evaluation_by_label.to_latex(index=False))